In [4]:
import sys
import os
sys.path.append('/root/capsule/code/beh_ephys_analysis')
from harp.clock import decode_harp_clock, align_timestamps_to_anchor_points
from open_ephys.analysis import Session
import datetime
from aind_ephys_rig_qc.temporal_alignment import search_harp_line
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib.colors import LinearSegmentedColormap
import pandas as pd
from pynwb import NWBFile, TimeSeries, NWBHDF5IO
from scipy.io import loadmat
from scipy.stats import zscore
import ast
from utils.plot_utils import combine_pdf_big

from open_ephys.analysis import Session
from pathlib import Path
import glob

import json
import seaborn as sns
from PyPDF2 import PdfMerger
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
import re
from aind_dynamic_foraging_basic_analysis.plot.plot_foraging_session import plot_foraging_session
from aind_dynamic_foraging_data_utils.nwb_utils import load_nwb_from_filename
from hdmf_zarr.nwb import NWBZarrIO
from utils.beh_functions import session_dirs, parseSessionID, load_model_dv, makeSessionDF, get_session_tbl, get_unit_tbl, get_history_from_nwb
from utils.ephys_functions import*
import pandas as pd
import pickle
import scipy.stats as stats
from joblib import Parallel, delayed
from multiprocessing import Pool
from functools import partial
import time
import spikeinterface as si
import shutil

In [5]:
# loop through all sessions
# count number of units pass qc and opto qc
# check if exist in longer than 100 sessions 
# check if exist in pre/post stimulation
# check if exist in anti-dromic stimulation
all_sessions = []
all_units = []
all_qc_pass_units = [] # default qc pass
all_real_units = [] # not noise, not artifact
all_tagged_units = []
in_behavior = []
opto_tagging_pre = []
opto_tagging_post = []
anti = []

df = pd.read_csv('/root/capsule/code/data_management/session_assets.csv')
session_ids = df['session_id'].values
session_ids = [session for session in session_ids[:-1] if isinstance(session, str)]
session_ids = [session for session in session_ids if '717121' not in session]

# p_resp_thresh = 0.5
# lat_resp_thresh = 0.02

target = 'soma'
for session in session_ids:
    session_dir = session_dirs(session)
    if os.path.exists(os.path.join(session_dir['beh_fig_dir'], f'{session}.nwb')):
        print(f'Processing {session}')
        if session_dir['curated_dir_curated'] is not None:
            data_type = 'curated'
        elif session_dir['curated_dir_raw'] is not None:
            data_type = 'raw'
        else:
            continue

        unit_tbl = get_unit_tbl(session, data_type)
        all_units.extend(unit_tbl['unit_id'].tolist())
        all_qc_pass_units.extend(unit_tbl['default_qc'].tolist())  # default qc pass
        all_real_units.extend(unit_tbl['real_unit'].tolist())
        all_tagged_units.extend(unit_tbl['tagged_loc'].tolist())  # tagged location (e.g. 'soma', 'axon', 'unspecified')
        all_sessions.extend([session]*len(unit_tbl))


        # session and opto information
        session_df = get_session_tbl(session)
        with open(os.path.join(session_dir[f'opto_dir_{data_type}'], f'{session}_opto_info_{target}.json')) as f:
            opto_info = json.load(f)
        opto_df = pd.read_csv(os.path.join(session_dir[f'opto_dir_{data_type}'], f'{session}_opto_session_{target}.csv'))
        if len(opto_df[opto_df['pre_post'] == 'pre'])>0:
            pre_end = np.max(opto_df[opto_df['pre_post'] == 'pre']['time'].values)
        else:
            pre_end = np.nan
        
        if len(opto_df[opto_df['pre_post'] == 'post'])>0:
            post_start = np.min(opto_df[opto_df['pre_post'] == 'post']['time'].values)
            post_end = np.max(opto_df[opto_df['pre_post'] == 'post']['time'].values)
        else:
            post_start = np.nan
            post_end = np.nan
            
        for unit_id in unit_tbl['unit_id'].values:
            curr_pre_opto = True
            curr_post_opto = True 
            curr_anti_opto = True  
            curr_in_beh = True
            unit = unit_tbl[unit_tbl['unit_id'] == unit_id]
            unit_drift = load_drift(session, unit_id, data_type=data_type)
            go_cue_times = session_df['goCue_start_time']
            if unit_drift is not None:
                if unit_drift['ephys_cut'][0] is not None:
                    if unit_drift['ephys_cut'][0] > pre_end - 2*60:
                        curr_pre_opto = False 
                    go_cue_times = go_cue_times[go_cue_times >= unit_drift['ephys_cut'][0]]         
                if unit_drift['ephys_cut'][1] is not None:
                    if unit_drift['ephys_cut'][1] < post_start + 2*60:
                        curr_post_opto = False
                    if unit_drift['ephys_cut'][1] < post_end + 2*60:
                        curr_anti_opto = False
                    go_cue_times = go_cue_times[go_cue_times <= unit_drift['ephys_cut'][1]]
            if len(go_cue_times) < 150:
                curr_in_beh = False
            opto_tagging_pre.append(curr_pre_opto)
            opto_tagging_post.append(curr_post_opto)
            anti.append(curr_anti_opto)
            in_behavior.append(curr_in_beh)

Processing behavior_716325_2024-05-31_10-31-14
There are multiple recordings in the curated nwb directory. Please specify the recording you would like to use.
Processing behavior_751004_2024-12-19_11-50-37
Processing behavior_751004_2024-12-20_13-26-11
Processing behavior_751004_2024-12-21_13-28-28
Processing behavior_751004_2024-12-22_13-09-17
Processing behavior_751004_2024-12-23_14-20-03
Processing behavior_751769_2025-01-16_11-32-05
Processing behavior_751769_2025-01-17_11-37-39
Processing behavior_751769_2025-01-18_10-15-25
Processing behavior_758017_2025-02-04_11-57-38
Processing behavior_758017_2025-02-05_11-42-34
Processing behavior_758017_2025-02-06_11-26-14
Processing behavior_758017_2025-02-07_14-11-08
Processing behavior_751766_2025-02-11_11-53-38
Processing behavior_751766_2025-02-13_11-31-21
Processing behavior_751766_2025-02-14_11-37-11
There is no nwb file in the curated directory.
Processing behavior_751766_2025-02-15_12-08-11
Processing behavior_751181_2025-02-25_12-1

In [8]:
# grab all tagged units and combine a pdf
data_type = 'curated'
target = 'soma'
selected_sessions = []
selected_units = []
all_pdf = []
for session, unit, tag, qc in zip(all_sessions, all_units, all_tagged_units, all_qc_pass_units):
    if tag and qc:
        session_dir = session_dirs(session)
        # # opto_file
        # all_pdfs = os.listdir(session_dir[f'opto_dir_fig_{data_type}'])
        # pdf = [pdf for pdf in all_pdfs if f'unit_{unit}_' in pdf and pdf.endswith('.pdf')][0]
        # pdf_path = os.path.join(session_dir[f'opto_dir_fig_{data_type}'], pdf)
        # # copy to new path
        # new_path_opto = os.path.join('/root/capsule/scratch/combined/opto_tagged', f'{session}_{pdf}')
        # _ = shutil.copy(pdf_path, new_path_opto)

        # selected_sessions.append(session)
        # selected_units.append(unit)
        # all_pdf.append(pdf_path)

        # # behavior file: go cue
        # all_pdfs = os.listdir(os.path.join(session_dir[f'ephys_fig_dir_{data_type}'], 'go_cue'))
        # pdf = [pdf for pdf in all_pdfs if f'unit_{unit}_' in pdf and pdf.endswith('.pdf')][0]
        # pdf_path = os.path.join(session_dir[f'ephys_fig_dir_{data_type}'], 'go_cue', pdf)
        # # copy to new path
        # new_path_ephys = os.path.join('/root/capsule/scratch/combined/behavior_tagged_go_cue', f'{session}_{pdf}')
        # _ = shutil.copy(pdf_path, new_path_ephys)

        # behavior file: go cue
        all_pdfs = os.listdir(os.path.join(session_dir[f'ephys_fig_dir_{data_type}'], 'response'))
        pdf = [pdf for pdf in all_pdfs if f'unit_{unit}_' in pdf and pdf.endswith('.pdf')][0]
        pdf_path = os.path.join(session_dir[f'ephys_fig_dir_{data_type}'], 'response', pdf)
        # copy to new path
        new_path_ephys = os.path.join('/root/capsule/scratch/combined/behavior_tagged_response', f'{session}_{pdf}')
        _ = shutil.copy(pdf_path, new_path_ephys)


In [7]:
opto_sessions = []
for session, unit, tag, qc in zip(all_sessions, all_units, all_tagged_units, all_qc_pass_units):
    if tag and qc:
        opto_sessions.append(session)

opto_sessions = list(set(opto_sessions))

for session in opto_sessions:
    session_dir = session_dirs(session)
    opto_summary_file = os.path.join(session_dir[f'opto_dir_{data_type}'], f"{session}_{data_type}_{target}_opto_summary.pdf")
    if os.path.exists(opto_summary_file):
        new_path = os.path.join('/root/capsule/scratch/combined/opto_summary', f'{session}_opto_summary.pdf')
        _ = shutil.copy(opto_summary_file, new_path)


In [8]:
# combine_pdf_big('/root/capsule/scratch/combined/opto_tagged', os.path.join('/root/capsule/scratch/combined', 'opto_tagged_combined.pdf'))
# combine_pdf_big('/root/capsule/scratch/combined/behavior_tagged_go_cue', os.path.join('/root/capsule/scratch/combined', 'behavior_ephys_tagged_go_cue_combined.pdf'))

# combine_pdf_big('/root/capsule/scratch/combined/behavior_tagged_response', os.path.join('/root/capsule/scratch/combined', 'behavior_ephys_tagged_response_combined.pdf'))
combine_pdf_big('/root/capsule/scratch/combined/opto_summary', os.path.join('/root/capsule/scratch/combined', 'opto_probe_summary.pdf'))

Processing 23 files in /root/capsule/scratch/combined/opto_summary


/opt/conda/lib/python3.10/site-packages/PIL/Image.py:3402: DecompressionBombWarning: Image size (97392750 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/PIL/Image.py:3402: DecompressionBombWarning: Image size (97392750 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/PIL/Image.py:3402: DecompressionBombWarning: Image size (97392750 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/PIL/Image.py:3402: DecompressionBombWarning: Image size (97392750 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/PIL/Image.py:3402: DecompressionBombWarning: Image size (97392750 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS 

Combined PDF saved as: /root/capsule/scratch/combined/opto_probe_summary.pdf
